## 📦 Step 1: Setup e Imports

In [2]:
# Imports
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports completos")

✅ Imports completos


## 📊 Step 2: Carregar os 3 Databases

In [3]:
# Para rodar no Colab, você precisa fazer upload ou montar o Google Drive
# Descomente uma das opções abaixo:

# OPÇÃO 1: Google Drive (recomendado)
# from google.colab import drive
# drive.mount('/content/drive')
# base_path = '/content/drive/My Drive/datalake'

# OPÇÃO 2: Local (se rodando localmente)
base_path = 'datalake'

print("📊 Carregando databases...")

players_df = pd.read_csv('players_complete_1995_2025.csv')
team_stats_df = pd.read_csv('teams_complete_1995_2025.csv')
squads_df = pd.read_csv('squads_complete.csv')

print(f"✅ Players: {len(players_df)} registros")
print(f"✅ Teams Stats: {len(team_stats_df)} registros")
print(f"✅ Squads: {len(squads_df)} registros")

📊 Carregando databases...
✅ Players: 86930 registros
✅ Teams Stats: 3206 registros
✅ Squads: 553 registros


## 🎯 Step 3: Selecionar Features para Clusterização

In [4]:
# Feature Columns - Do que vêm e o que significam

FEATURE_COLUMNS = {
    'performance': [
        'Performance_Gls',      # Gols marcados
        'Performance_Ast',      # Assistências
        'Performance_G+A',      # Gols + Assistências
        'Performance_G-PK',     # Gols sem pênaltis
        'Performance_PK',       # Pênaltis
        'Performance_CrdY',     # Cartões amarelos (disciplina)
        'Performance_CrdR',     # Cartões vermelhos
    ],
    'expected': [
        'Expected_xG',          # xG - qualidade de chutes
        'Expected_xAG',         # xA - qualidade de passes de gol
        'Expected_npxG+xAG',    # xG + xA sem pênaltis
    ],
    'per_90': [
        'Per_90_Minutes_Gls',   # Gols por 90 min
        'Per_90_Minutes_Ast',   # Assist por 90 min
        'Per_90_Minutes_G+A',   # G+A por 90 min
        'Per_90_Minutes_xG',    # xG por 90 min
        'Per_90_Minutes_xAG',   # xA por 90 min
        'Per_90_Minutes_xG+xAG',# xG+xA por 90 min
    ],
    'playing_time': [
        'Playing_Time_MP',      # Matches Played
        'Playing_Time_Starts',  # Partidas como titular
        'Playing_Time_Min',     # Minutos totais
        'Playing_Time_90s',     # Partidas equivalentes (90min)
    ],
    'progression': [
        'Progression_PrgC',     # Carregadas progressivas
        'Progression_PrgP',     # Passes progressivos
        'Progression_PrgR',     # Corridas progressivas
    ],
    'basic': [
        'age',                  # Idade
    ]
}

# Selecionar features que existem
all_features = []
for category, features in FEATURE_COLUMNS.items():
    available = [col for col in features if col in players_df.columns]
    all_features.extend(available)
    print(f"  {category.upper()}: {len(available)}/{len(features)} features")

print(f"\n✅ Total de features para clusterização: {len(all_features)}")

  PERFORMANCE: 7/7 features
  EXPECTED: 3/3 features
  PER_90: 6/6 features
  PLAYING_TIME: 4/4 features
  PROGRESSION: 3/3 features
  BASIC: 1/1 features

✅ Total de features para clusterização: 24


## 🔍 Step 4: Grid Search - Encontrar Número Ótimo de Clusters

In [5]:
# Preparar dados
x = players_df[all_features].fillna(0)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(x)

# Grid Search
print("🔍 Grid Search: Testando diferentes números de clusters...\n")

silhouette_scores = {}
inertias = {}

for n_clusters in range(2, 16):
    kmeans_test = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans_test.fit_predict(X_scaled)

    silhouette = silhouette_score(X_scaled, cluster_labels)
    inertia = kmeans_test.inertia_

    silhouette_scores[n_clusters] = silhouette
    inertias[n_clusters] = inertia

    print(f"  n_clusters={n_clusters:2d} | Silhouette: {silhouette:.4f} | Inertia: {inertia:.2f}")

# Encontrar melhor
optimal_clusters = max(silhouette_scores, key=silhouette_scores.get)
print(f"\n✅ Número ótimo de clusters: {optimal_clusters} (silhouette: {silhouette_scores[optimal_clusters]:.4f})")

🔍 Grid Search: Testando diferentes números de clusters...

  n_clusters= 2 | Silhouette: 0.3252 | Inertia: 1664918.76
  n_clusters= 3 | Silhouette: 0.3229 | Inertia: 1351449.07
  n_clusters= 4 | Silhouette: 0.3142 | Inertia: 1221565.67
  n_clusters= 5 | Silhouette: 0.3207 | Inertia: 1127033.10
  n_clusters= 6 | Silhouette: 0.3234 | Inertia: 1044076.19
  n_clusters= 7 | Silhouette: 0.3233 | Inertia: 993055.60
  n_clusters= 8 | Silhouette: 0.3109 | Inertia: 942528.29
  n_clusters= 9 | Silhouette: 0.3109 | Inertia: 889620.90
  n_clusters=10 | Silhouette: 0.2006 | Inertia: 854001.89
  n_clusters=11 | Silhouette: 0.1962 | Inertia: 825400.46
  n_clusters=12 | Silhouette: 0.2030 | Inertia: 783819.31
  n_clusters=13 | Silhouette: 0.2064 | Inertia: 749181.32
  n_clusters=14 | Silhouette: 0.2082 | Inertia: 721260.28
  n_clusters=15 | Silhouette: 0.2054 | Inertia: 696211.28

✅ Número ótimo de clusters: 2 (silhouette: 0.3252)


## 🎮 Step 5: Clusterização Final de Jogadores

In [6]:
# Clusterizar com número ótimo
kmeans = KMeans(n_clusters=optimal_clusters, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_scaled)

print(f"📈 Distribuição de clusters:\n")
for i in range(optimal_clusters):
    count = (clusters == i).sum()
    print(f"  Cluster {i}: {count:5d} jogadores")

# Adicionar ao dataframe
players_df['player_cluster'] = clusters
players_df['player_vector'] = list(X_scaled)

print(f"\n✅ Clusterização completa!")

📈 Distribuição de clusters:

  Cluster 0: 53624 jogadores
  Cluster 1: 33306 jogadores

✅ Clusterização completa!


## 👥 Step 6: Calcular Vetores de Times (Agregado dos Jogadores)

In [9]:
def calculate_team_vector(team_name, squads_df, players_df):
    """
    Calcula vetor tático de um time como MÉDIA dos embeddings dos jogadores
    Isso é MUITO mais correto que usar estatísticas brutas de time
    """
    # Pegar jogadores do time do squad
    team_players = squads_df[squads_df['team'].str.lower() == team_name.lower()]

    if len(team_players) == 0:
        return None

    # Encontrar vetores dos jogadores
    team_vectors = []
    matched_players = 0

    for idx, squad_player in team_players.iterrows():
        # Buscar jogador no dataframe principal
        player_match = players_df[
            (players_df['player'].str.lower() == str(squad_player['player_name']).lower())
        ]

        if len(player_match) > 0:
            player = player_match.iloc[0]
            if 'player_vector' in player and player['player_vector'] is not None:
                team_vectors.append(player['player_vector'])
                matched_players += 1

    if len(team_vectors) == 0:
        return None

    # Retornar média ponderada
    team_vector = np.mean(team_vectors, axis=0)
    return team_vector, matched_players

print("📐 Gerando team_vectors como média ponderada dos player_vectors...\n")

# Calcular team_vectors
teams_vectors_dict = {}
teams_info = []

unique_teams = squads_df['team'].unique()

for team_name in unique_teams:
    result = calculate_team_vector(team_name, squads_df, players_df)
    if result is not None:
        team_vector, matched = result
        teams_vectors_dict[team_name] = team_vector
        teams_info.append({
            'team': team_name,
            'matched_players': matched,
            'team_vector': team_vector
        })

teams_vectors_df = pd.DataFrame(teams_info)
print(f"✅ Team vectors calculados para {len(teams_vectors_df)} times")
print(f"   Média de jogadores encontrados: {teams_vectors_df['matched_players'].mean():.1f}")

📐 Gerando team_vectors como média ponderada dos player_vectors...

✅ Team vectors calculados para 16 times
   Média de jogadores encontrados: 27.8


## 🏆 Step 7: Clusterizar Times por Estilo Tático

In [10]:
print("🏆 CLUSTERIZANDO TIMES POR ESTILO TÁTICO\n")
print("⚠️  IMPORTANTE: Features de ESTILO, não de PRODUÇÃO")
print("   Isso permite identificar: defensivo, pressão alta, transição, etc.\n")

# Features de ESTILO (não produção)
team_style_features = [
    'Poss',         # % Posse de bola
    'Press',        # Pressões por 90
    'Tkl+Int',      # Desarmes + Interceptações
    'Clr',          # Limpezas
    'PrgP',         # Passes progressivos
    'PrgC',         # Carregadas progressivas
    'PrgR',         # Corridas progressivas
    'Cmp%',         # % Passes completos
    'KP',           # Key passes
    'Succ%',        # % Sucesso em dribles
]

available_style_features = [col for col in team_style_features if col in team_stats_df.columns]
print(f"✅ Features de estilo disponíveis: {len(available_style_features)}/{len(team_style_features)}")
print(f"   Features: {available_style_features}\n")

if len(available_style_features) > 0:
    # Preparar dados
    team_style_data = team_stats_df[available_style_features].fillna(0)

    # Normalizar
    team_style_scaler = StandardScaler()
    team_style_scaled = team_style_scaler.fit_transform(team_style_data)

    # Grid Search para times
    print("🔍 Grid Search: Testando diferentes números de clusters de times...\n")

    team_silhouette_scores = {}

    for n_clusters_team in range(2, min(8, len(team_stats_df))):
        kmeans_teams_test = KMeans(n_clusters=n_clusters_team, random_state=42, n_init=10)
        team_cluster_labels = kmeans_teams_test.fit_predict(team_style_scaled)

        team_silhouette = silhouette_score(team_style_scaled, team_cluster_labels)
        team_silhouette_scores[n_clusters_team] = team_silhouette

        print(f"  n_clusters={n_clusters_team:2d} | Silhouette: {team_silhouette:.4f}")

    optimal_team_clusters = max(team_silhouette_scores, key=team_silhouette_scores.get)
    print(f"\n✅ Número ótimo de clusters de times: {optimal_team_clusters}")
    print(f"   Silhouette: {team_silhouette_scores[optimal_team_clusters]:.4f}\n")

    # Clusterizar
    kmeans_teams = KMeans(n_clusters=optimal_team_clusters, random_state=42, n_init=10)
    team_clusters = kmeans_teams.fit_predict(team_style_scaled)

    team_stats_df['team_cluster'] = team_clusters
    team_stats_df['team_vector'] = list(team_style_scaled)

    print(f"📊 Distribuição de clusters de times (por estilo):\n")
    for i in range(optimal_team_clusters):
        cluster_teams = team_stats_df[team_stats_df['team_cluster'] == i]
        count = len(cluster_teams)
        teams_list = list(cluster_teams['Squad'].head(5).values)

        avg_poss = cluster_teams['Poss'].mean() if 'Poss' in cluster_teams.columns else 0
        avg_press = cluster_teams['Press'].mean() if 'Press' in cluster_teams.columns else 0

        print(f"  Cluster {i}: {count:2d} times")
        print(f"    Times: {', '.join(teams_list[:3])}")
        print(f"    Estilo: Posse {avg_poss:.1f}%, Pressão {avg_press:.1f}\n")
else:
    print("⚠️  Features de estilo não encontradas. Usando vetores de jogadores.\n")

🏆 CLUSTERIZANDO TIMES POR ESTILO TÁTICO

⚠️  IMPORTANTE: Features de ESTILO, não de PRODUÇÃO
   Isso permite identificar: defensivo, pressão alta, transição, etc.

✅ Features de estilo disponíveis: 0/10
   Features: []

⚠️  Features de estilo não encontradas. Usando vetores de jogadores.



## 💼 Step 8: Recomendações de Transferência

In [19]:
def get_current_club(player_name, squads_df):
    """
    Busca o time ATUAL do jogador no squads_df (temporada 2025-26)
    """
    player_squad = squads_df[squads_df['player_name'].str.lower() == player_name.lower()]

    if len(player_squad) == 0:
        # Tentar com coluna alternativa
        player_squad = squads_df[squads_df['Player'].str.lower() == player_name.lower()]

    if len(player_squad) > 0:
        return player_squad.iloc[0].get('team', 'Unknown')

    return 'Unknown'

def recommend_transfers(team_name, squads_df, players_df, teams_vectors_dict, top_k=7):
    """
    Recomenda os melhores jogadores disponíveis para um time
    baseado em compatibilidade vetorial
    """

    if team_name not in teams_vectors_dict:
        return None

    team_vector = teams_vectors_dict[team_name]

    # Jogadores no time atual
    team_squad = squads_df[squads_df['team'].str.lower() == team_name.lower()]
    squad_player_names = set()

    for idx, squad_player in team_squad.iterrows():
        player_name = squad_player.get('player_name', squad_player.get('Player', '')).lower()
        squad_player_names.add(player_name)

    # Calcular match_score
    recommendations = []

    for idx, player in players_df.iterrows():
        player_name = str(player.get('player', player.get('Name', ''))).lower()

        # Pular se já está no time ou é goleiro
        if player_name in squad_player_names:
            continue

        if str(player.get('pos', '')).upper() == 'GK':
            continue

        if 'player_vector' in player and player['player_vector'] is not None:
            try:
                # Buscar time ATUAL do jogador no squads_df (2025-26)
                current_club = get_current_club(player_name, squads_df)

                sim = cosine_similarity([team_vector], [player['player_vector']])[0][0]
                recommendations.append({
                    'Jogador': player_name.title(),
                    'Posição': player.get('pos', 'Unknown'),
                    'Cluster': player.get('player_cluster', -1),
                    'Idade': int(player.get('age', 0)),
                    'Time_Atual': current_club.title() if current_club else 'Unknown',
                    'Gols_Carreira': float(player.get('Performance_Gls', 0)),
                    'Match_Score': round(sim, 4),
                })
            except:
                pass

    if len(recommendations) == 0:
        return None

    df_rec = pd.DataFrame(recommendations).sort_values('Match_Score', ascending=False)
    return df_rec.head(top_k)

print("💼 SISTEMA DE RECOMENDAÇÃO DE TRANSFERÊNCIA\n")
print("📋 Recomendações de Transferência (Top 7 jogadores disponíveis)")
print("=" * 70 + "\n")

teams_to_recommend = list(teams_vectors_dict.keys())[:5]

for team in teams_to_recommend:
    print(f"🔍 {team.upper()}")
    print("   Melhores encaixes para reforço:\n")

    recommendations = recommend_transfers(team, squads_df, players_df, teams_vectors_dict, top_k=5)

    if recommendations is not None:
        for idx, row in recommendations.iterrows():
            score = row['Match_Score']
            pos = row['Posição']
            age = int(row['Idade'])
            current = row['Time_Atual']
            player = row['Jogador']

            if score > 0.85:
                icon = "🟢"
            elif score > 0.80:
                icon = "🟡"
            else:
                icon = "🟠"

            print(f"   {icon} {player:25s} | {pos:3s} | {age:2d}y | {current:20s} | {score:.4f}")

    print()

💼 SISTEMA DE RECOMENDAÇÃO DE TRANSFERÊNCIA

📋 Recomendações de Transferência (Top 7 jogadores disponíveis)

🔍 BAYERN MUNICH
   Melhores encaixes para reforço:

   🟢 Youssouf Fofana           | MF  | 19y | Ac Milan             | 0.8883
   🟢 Rico Lewis                | DF,MF | 18y | Manchester City      | 0.8525
   🟢 Warren Zaïre-Emery        | MF,DF | 16y | Psg                  | 0.8515
   🟡 Pierre Kalulu             | DF  | 20y | Juventus             | 0.8471
   🟡 Angel Gomes               | MF,FW | 20y | Marseille            | 0.8102

🔍 BORUSSIA DORTMUND
   Melhores encaixes para reforço:

   🟢 Marc Pubill               | DF,MF | 18y | Atlético Madrid      | 0.9096
   🟢 Rayan Cherki              | FW,MF | 17y | Manchester City      | 0.8702
   🟢 Pierre Kalulu             | DF  | 20y | Juventus             | 0.8607
   🟡 Kassoum Ouattara          | DF,FW | 18y | Monaco               | 0.8499
   🟡 Rico Lewis                | DF,MF | 18y | Manchester City      | 0.8405

🔍 RB LEIPZIG
   Me

## 📊 Step 9: Compatibilidade de Squad Atual

In [13]:
def analyze_squad_compatibility(team_name, squads_df, players_df, teams_vectors_dict):
    """
    Analisa compatibilidade dos jogadores ATUAIS com o estilo do time
    """

    if team_name not in teams_vectors_dict:
        return None

    team_vector = teams_vectors_dict[team_name]
    team_squad = squads_df[squads_df['team'].str.lower() == team_name.lower()]

    compatibilities = []

    for idx, squad_player in team_squad.iterrows():
        player_name = squad_player.get('player_name', squad_player.get('Player', '')).lower()
        player_data = players_df[players_df['player'].str.lower() == player_name]

        if len(player_data) > 0:
            player = player_data.iloc[0]

            if 'player_vector' in player and player['player_vector'] is not None:
                try:
                    sim = cosine_similarity([team_vector], [player['player_vector']])[0][0]
                    compatibilities.append({
                        'Jogador': player_name.title(),
                        'Posição': player.get('pos', 'Unknown'),
                        'Cluster': player.get('player_cluster', -1),
                        'Fit_Score': round(sim, 4),
                    })
                except:
                    pass

    if len(compatibilities) == 0:
        return None

    df_compat = pd.DataFrame(compatibilities).sort_values('Fit_Score', ascending=False)
    return df_compat

print("📊 ANÁLISE DE COMPATIBILIDADE - SQUAD ATUAL\n")
print("=" * 70 + "\n")

for team in teams_to_recommend:
    print(f"{team}:")
    compat = analyze_squad_compatibility(team, squads_df, players_df, teams_vectors_dict)

    if compat is not None:
        print("\n   ✅ Melhor Fit (3 primeiros):")
        for idx, row in compat.head(3).iterrows():
            print(f"      {row['Jogador']:25s} | {row['Posição']:3s} | Fit: {row['Fit_Score']:.4f}")

        print("\n   ⚠️  Pior Fit (3 últimos):")
        for idx, row in compat.tail(3).iterrows():
            print(f"      {row['Jogador']:25s} | {row['Posição']:3s} | Fit: {row['Fit_Score']:.4f}")

    print()

📊 ANÁLISE DE COMPATIBILIDADE - SQUAD ATUAL


Bayern Munich:

   ✅ Melhor Fit (3 primeiros):
      Konrad Laimer             | DF,MF | Fit: 0.6395
      Michael Olise             | FW,MF | Fit: 0.5953
      Aleksandar Pavlovic       | MF  | Fit: 0.5776

   ⚠️  Pior Fit (3 últimos):
      Joshua Kimmich            | DF,MF | Fit: 0.2617
      Manuel Neuer              | GK  | Fit: 0.1378
      Raphaël Guerreiro         | DF,MF | Fit: 0.1102

Borussia Dortmund:

   ✅ Melhor Fit (3 primeiros):
      Daniel Svensson           | DF,FW | Fit: 0.6660
      Felix Nmecha              | MF,FW | Fit: 0.6444
      Julian Ryerson            | DF,MF | Fit: 0.6203

   ⚠️  Pior Fit (3 últimos):
      Alexander Meyer           | DF  | Fit: -0.0822
      Marcel Sabitzer           | FW,MF | Fit: -0.1157
      Patrick Drewes            | GK  | Fit: -0.7293

RB Leipzig:

   ✅ Melhor Fit (3 primeiros):
      Antonio Nusa              | MF,DF | Fit: 0.7357
      Amadou Haidara            | MF  | Fit: 0.7301
  

## ⚖️ Step 10: Desbalanceamentos de Cluster nos Times

In [14]:
def analyze_cluster_balance(team_name, squads_df, players_df):
    """
    Analisa distribuição de clusters no time
    """

    team_squad = squads_df[squads_df['team'].str.lower() == team_name.lower()]

    players_in_team = []
    for idx, squad_player in team_squad.iterrows():
        player_name = squad_player.get('player_name', squad_player.get('Player', '')).lower()
        player_data = players_df[players_df['player'].str.lower() == player_name]

        if len(player_data) > 0:
            players_in_team.append(player_data.iloc[0])

    if len(players_in_team) == 0:
        return None

    team_players_df = pd.DataFrame(players_in_team)

    cluster_dist = team_players_df['player_cluster'].value_counts().sort_index()
    global_dist = players_df['player_cluster'].value_counts(normalize=True).sort_index()

    return {
        'team': team_name,
        'local': cluster_dist.to_dict(),
        'global': global_dist.to_dict(),
        'total': len(team_players_df),
    }

print("⚖️  ANÁLISE DE DESBALANCEAMENTOS DE CLUSTER\n")
print("=" * 70 + "\n")

for team in teams_to_recommend:
    analysis = analyze_cluster_balance(team, squads_df, players_df)

    if analysis:
        print(f"{team} (Total: {analysis['total']} jogadores)")
        print("   Cluster | Local | Global | Diff")

        all_clusters = set(list(analysis['local'].keys()) + list(analysis['global'].keys()))

        for cluster in sorted(all_clusters):
            local = analysis['local'].get(cluster, 0)
            global_pct = analysis['global'].get(cluster, 0) * 100
            local_pct = (local / analysis['total'] * 100) if analysis['total'] > 0 else 0
            diff = local_pct - global_pct

            if diff > 5:
                icon = "🔴"
            elif diff < -5:
                icon = "🔵"
            else:
                icon = "⚪"

            print(f"   {icon} {cluster:7d} | {local_pct:5.1f}% | {global_pct:6.1f}% | {diff:+6.1f}%")

    print()

⚖️  ANÁLISE DE DESBALANCEAMENTOS DE CLUSTER


Bayern Munich (Total: 20 jogadores)
   Cluster | Local | Global | Diff
   ⚪       0 |  65.0% |   61.7% |   +3.3%
   ⚪       1 |  35.0% |   38.3% |   -3.3%

Borussia Dortmund (Total: 24 jogadores)
   Cluster | Local | Global | Diff
   🔴       0 |  70.8% |   61.7% |   +9.1%
   🔵       1 |  29.2% |   38.3% |   -9.1%

RB Leipzig (Total: 21 jogadores)
   Cluster | Local | Global | Diff
   ⚪       0 |  57.1% |   61.7% |   -4.5%
   ⚪       1 |  42.9% |   38.3% |   +4.5%

Atlético Madrid (Total: 21 jogadores)
   Cluster | Local | Global | Diff
   🔴       0 |  71.4% |   61.7% |   +9.7%
   🔵       1 |  28.6% |   38.3% |   -9.7%

Barcelona (Total: 17 jogadores)
   Cluster | Local | Global | Diff
   ⚪       0 |  58.8% |   61.7% |   -2.9%
   ⚪       1 |  41.2% |   38.3% |   +2.9%



## 🎯 Step 11: Ranking com Contexto Tático

In [20]:
def contextual_ranking(team_name, squads_df, players_df, teams_vectors_dict, team_stats_df, weight_tactical=0.6):
    """
    Ranking avançado que considera:
    - Match score vetorial (40%)
    - Compatibilidade de cluster (60%)
    """

    if team_name not in teams_vectors_dict:
        return None

    team_vector = teams_vectors_dict[team_name]
    team_squad = squads_df[squads_df['team'].str.lower() == team_name.lower()]

    squad_player_names = set()
    for idx, squad_player in team_squad.iterrows():
        player_name = squad_player.get('player_name', squad_player.get('Player', '')).lower()
        squad_player_names.add(player_name)

    # Encontrar cluster dominante
    team_cluster_dist = {}
    for idx, squad_player in team_squad.iterrows():
        player_name = squad_player.get('player_name', squad_player.get('Player', '')).lower()
        player_data = players_df[players_df['player'].str.lower() == player_name]

        if len(player_data) > 0:
            cluster = player_data.iloc[0].get('player_cluster', -1)
            team_cluster_dist[cluster] = team_cluster_dist.get(cluster, 0) + 1

    if not team_cluster_dist:
        return None

    primary_clusters = sorted(team_cluster_dist.items(), key=lambda x: x[1], reverse=True)[:2]
    primary_cluster_ids = [c[0] for c in primary_clusters]

    # Score contextual
    recommendations = []

    for idx, player in players_df.iterrows():
        player_name = str(player.get('player', player.get('Name', ''))).lower()

        if player_name in squad_player_names:
            continue

        if str(player.get('pos', '')).upper() == 'GK':
            continue

        if 'player_vector' in player and player['player_vector'] is not None:
            try:
                # Buscar time ATUAL do jogador no squads_df (2025-26)
                current_club = get_current_club(player_name, squads_df)

                vector_score = cosine_similarity([team_vector], [player['player_vector']])[0][0]
                player_cluster = player.get('player_cluster', -1)
                tactical_score = 1.0 if player_cluster in primary_cluster_ids else 0.6
                final_score = (vector_score * (1 - weight_tactical)) + (tactical_score * weight_tactical)

                recommendations.append({
                    'Jogador': player_name.title(),
                    'Posição': player.get('pos', 'Unknown'),
                    'Cluster': player_cluster,
                    'Idade': int(player.get('age', 0)),
                    'Time': current_club.title(),
                    'Vector_Score': round(vector_score, 4),
                    'Tactical_Fit': round(tactical_score, 2),
                    'Final_Score': round(final_score, 4),
                })
            except:
                pass

    if len(recommendations) == 0:
        return None

    df_rec = pd.DataFrame(recommendations).sort_values('Final_Score', ascending=False)
    return df_rec.head(7)

print("🎯 RANKING COM CONTEXTO TÁTICO\n")
print("=" * 70 + "\n")
print("Formula: Final_Score = (Vector_Score × 0.4) + (Tactical_Fit × 0.6)\n")

for team in teams_to_recommend[:3]:
    print(f"{team}:")
    rec = contextual_ranking(team, squads_df, players_df, teams_vectors_dict, team_stats_df)

    if rec is not None:
        print("   Jogador             | Pos | Idade | Vector | Tático | Final")
        for idx, row in rec.iterrows():
            print(f"   {row['Jogador']:20s} | {row['Posição']:3s} | {row['Idade']:5d} | {row['Vector_Score']:.4f} | {row['Tactical_Fit']:.2f}  | {row['Final_Score']:.4f}")

    print()

🎯 RANKING COM CONTEXTO TÁTICO


Formula: Final_Score = (Vector_Score × 0.4) + (Tactical_Fit × 0.6)

Bayern Munich:
   Jogador             | Pos | Idade | Vector | Tático | Final
   Youssouf Fofana      | MF  |    19 | 0.8883 | 1.00  | 0.9553
   Rico Lewis           | DF,MF |    18 | 0.8525 | 1.00  | 0.9410
   Warren Zaïre-Emery   | MF,DF |    16 | 0.8515 | 1.00  | 0.9406
   Pierre Kalulu        | DF  |    20 | 0.8471 | 1.00  | 0.9388
   Angel Gomes          | MF,FW |    20 | 0.8102 | 1.00  | 0.9241
   Fabio Miretti        | MF  |    19 | 0.8063 | 1.00  | 0.9225
   Gavi                 | MF,FW |    18 | 0.7997 | 1.00  | 0.9199

Borussia Dortmund:
   Jogador             | Pos | Idade | Vector | Tático | Final
   Marc Pubill          | DF,MF |    18 | 0.9096 | 1.00  | 0.9638
   Rayan Cherki         | FW,MF |    17 | 0.8702 | 1.00  | 0.9481
   Pierre Kalulu        | DF  |    20 | 0.8607 | 1.00  | 0.9443
   Kassoum Ouattara     | DF,FW |    18 | 0.8499 | 1.00  | 0.9400
   Rico Lewis        

## 💾 Step 12: Salvar Resultados

In [21]:
print("💾 SALVANDO RESULTADOS\n")
print("=" * 70 + "\n")

# Salvar players com clusters
players_output = players_df[['player', 'Club', 'pos', 'age', 'player_cluster']].copy()
players_output.to_csv('players_clustered.csv', index=False)
print("✅ Saved: players_clustered.csv")

# Salvar análise de clusters de jogadores
cluster_analysis = players_df.groupby('player_cluster').agg({
    'player': 'count',
    'age': 'mean',
    'Performance_Gls': 'mean',
    'Performance_Ast': 'mean',
    'pos': lambda x: x.value_counts().index[0] if len(x) > 0 else 'Unknown'
}).round(2)
cluster_analysis.columns = ['Total_Players', 'Avg_Age', 'Avg_Gls', 'Avg_Ast', 'Top_Position']
cluster_analysis.to_csv('player_clusters_analysis.csv')
print("✅ Saved: player_clusters_analysis.csv")

# Salvar análise de clusters de times
if 'team_cluster' in team_stats_df.columns:
    team_output = team_stats_df[['Squad', 'team_cluster']].copy()
    team_output.to_csv('teams_clustered.csv', index=False)
    print("✅ Saved: teams_clustered.csv")

print("\n" + "=" * 70)

💾 SALVANDO RESULTADOS


✅ Saved: players_clustered.csv
✅ Saved: player_clusters_analysis.csv



## 🎉 Resumo Final

In [22]:
print("\n🎉 AI FOOTBALL SCOUT - ANÁLISE COMPLETA!\n")
print("=" * 70)
print(f"✅ Jogadores clusterizados: {len(players_df):,}")
print(f"✅ Clusters de jogadores: {optimal_clusters}")
print(f"✅ Times com vetores: {len(teams_vectors_dict)}")
if 'team_cluster' in team_stats_df.columns:
    print(f"✅ Clusters de times: {optimal_team_clusters}")
print(f"✅ Features por jogador: {len(all_features)}")
print("=" * 70)
print("\n📌 Você conseguiu:")
print("   1. ✅ Clusterizar jogadores por perfil tático")
print("   2. ✅ Gerar vetores de times (agregado de jogadores)")
print("   3. ✅ Encontrar as melhores transferências por similarity")
print("   4. ✅ Analisar compatibilidade de squad atual")
print("   5. ✅ Identificar desbalanceamentos de cluster nos times")
print("   6. ✅ Ranking com contexto tático (vector + cluster)")
print("\n🚀 Próximos passos:")
print("   - Exportar CSVs com as recomendações")
print("   - Criar dashboard no Power BI")
print("   - Refinar pesos do ranking (atual: 60% táctico)")
print("="*70)


🎉 AI FOOTBALL SCOUT - ANÁLISE COMPLETA!

✅ Jogadores clusterizados: 86,930
✅ Clusters de jogadores: 2
✅ Times com vetores: 16
✅ Features por jogador: 24

📌 Você conseguiu:
   1. ✅ Clusterizar jogadores por perfil tático
   2. ✅ Gerar vetores de times (agregado de jogadores)
   3. ✅ Encontrar as melhores transferências por similarity
   4. ✅ Analisar compatibilidade de squad atual
   5. ✅ Identificar desbalanceamentos de cluster nos times
   6. ✅ Ranking com contexto tático (vector + cluster)

🚀 Próximos passos:
   - Exportar CSVs com as recomendações
   - Criar dashboard no Power BI
   - Refinar pesos do ranking (atual: 60% táctico)


In [23]:
# ============================================================================
# Step 13: Exportar CSVs Otimizados para Power BI
# ============================================================================

print("📤 EXPORTANDO DADOS PARA POWER BI\n")
print("=" * 70 + "\n")

# -------------------------------------------------------------------------
# CSV 1: TRANSFER RECOMMENDATIONS (Fato)
# -------------------------------------------------------------------------

all_recommendations = []

for team in teams_vectors_dict.keys():
    # Recomendações contextuais
    rec = contextual_ranking(team, squads_df, players_df, teams_vectors_dict, team_stats_df)

    if rec is not None:
        rec['recommended_to_team'] = team
        rec['recommendation_date'] = pd.Timestamp.now().strftime('%Y-%m-%d')
        rec['player_id'] = range(len(all_recommendations), len(all_recommendations) + len(rec))

        all_recommendations.append(rec)

# Consolidar
transfer_recommendations_df = pd.concat(all_recommendations, ignore_index=True)

# Renomear colunas para padrão BI
transfer_recommendations_df = transfer_recommendations_df.rename(columns={
    'Jogador': 'player_name',
    'Posição': 'position',
    'Idade': 'age',
    'Time': 'current_club',
    'Cluster': 'player_cluster',
    'Vector_Score': 'vector_score',
    'Tactical_Fit': 'tactical_fit',
    'Final_Score': 'final_score',
})

# Selecionar colunas finais
transfer_recommendations_df = transfer_recommendations_df[[
    'player_id', 'player_name', 'current_club', 'position', 'age',
    'recommended_to_team', 'vector_score', 'tactical_fit', 'final_score',
    'player_cluster', 'recommendation_date'
]]

transfer_recommendations_df.to_csv('transfer_recommendations.csv', index=False)
print(f"✅ CSV 1: transfer_recommendations.csv")
print(f"   Registros: {len(transfer_recommendations_df):,}")
print(f"   Colunas: {list(transfer_recommendations_df.columns)}\n")

# -------------------------------------------------------------------------
# CSV 2: CLUSTERS METADATA (Dimensão)
# -------------------------------------------------------------------------

# Clusters de Jogadores
player_cluster_metadata = players_df.groupby('player_cluster').agg({
    'player': 'count',
    'age': 'mean',
    'Performance_Gls': 'mean',
    'Performance_Ast': 'mean',
    'pos': lambda x: x.value_counts().index[0] if len(x) > 0 else 'Unknown',
}).round(2)

player_cluster_metadata.columns = ['total_players', 'avg_age', 'avg_goals', 'avg_assists', 'top_position']
player_cluster_metadata['cluster_type'] = 'player'
player_cluster_metadata['cluster_id'] = player_cluster_metadata.index

# Adicionar descrições táticas (exemplos - ajustar conforme seus dados)
tactical_profiles_map = {
    0: 'Centroavante Clássico',
    1: 'Médio Criativo',
    2: 'Defensor Central',
    3: 'Ponta Rápida',
    4: 'Volante Defensivo',
    5: 'Lateral Ofensivo',
    6: 'Meia Armador',
    7: 'Segundo Atacante',
}

player_cluster_metadata['tactical_profile'] = player_cluster_metadata['cluster_id'].map(
    lambda x: tactical_profiles_map.get(x, f'Cluster {x}')
)

# Clusters de Times (se existirem)
if 'team_cluster' in team_stats_df.columns:
    team_cluster_metadata = team_stats_df.groupby('team_cluster').agg({
        'Squad': 'count',
        'Performance_Gls': 'mean',
        'Performance_Ast': 'mean',
    }).round(2)

    team_cluster_metadata.columns = ['total_teams', 'avg_goals', 'avg_assists']
    team_cluster_metadata['cluster_type'] = 'team'
    team_cluster_metadata['cluster_id'] = team_cluster_metadata.index + 100  # Offset para não conflitar
    team_cluster_metadata['tactical_profile'] = team_cluster_metadata.index.map({
        0: 'Ataque Rápido',
        1: 'Bloco Baixo',
        2: 'Pressão Alta',
        3: 'Transição',
    })

    # Consolidar
    clusters_metadata_df = pd.concat([player_cluster_metadata, team_cluster_metadata], ignore_index=True)
else:
    clusters_metadata_df = player_cluster_metadata

clusters_metadata_df.to_csv('clusters_metadata.csv', index=False)
print(f"✅ CSV 2: clusters_metadata.csv")
print(f"   Registros: {len(clusters_metadata_df):,}")
print(f"   Colunas: {list(clusters_metadata_df.columns)}\n")

print("=" * 70)
print("🎉 CSVs prontos para Power BI!")
print("=" * 70)

📤 EXPORTANDO DADOS PARA POWER BI


✅ CSV 1: transfer_recommendations.csv
   Registros: 112
   Colunas: ['player_id', 'player_name', 'current_club', 'position', 'age', 'recommended_to_team', 'vector_score', 'tactical_fit', 'final_score', 'player_cluster', 'recommendation_date']

✅ CSV 2: clusters_metadata.csv
   Registros: 2
   Colunas: ['total_players', 'avg_age', 'avg_goals', 'avg_assists', 'top_position', 'cluster_type', 'cluster_id', 'tactical_profile']

🎉 CSVs prontos para Power BI!
